In [1]:
# Import packages
import os
import random
from PIL import Image
from torch import nn
import torch 
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import json 
from src.config import PROCESSED_DATA_DIR, RAW_DATA_DIR
import torch.optim as optim

from model_architectures.v01 import HHP01
from model_architectures.v02 import HHP02

from utils import train_validate_model

image_path = PROCESSED_DATA_DIR / "pizza_hamburger_hotdog_20_percent"
train_dir = image_path / 'train' 
test_dir = image_path / 'test'
valid_dir = image_path / 'valid'

device ='cuda' if torch.cuda.is_available() else 'cpu'

2024-06-21 21:42:50.495 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Git\hamburger-hotdog-pizza-classifier


In [2]:
config_path = 'params.json'

with open(config_path, 'r') as config_file:
   config = json.load(config_file)
#print(json.dumps(config, indent=2))

In [3]:
# Load hyperparams from params.json

learning_rate = config['model_params']['learning_rate']
batch_size = config['model_params']['batch_size']
num_epochs = config['model_params']['num_epochs']
dropout_rate = config['model_params']['dropout_rate']
optimizer = config['model_params']['optimizer']
loss_function = config['model_params']['loss_function']
metrics = config['model_params']['metrics']
output_shape = config['model_params']['output_shape']

In [4]:
# Prepare data into dataloader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
])

train_data = ImageFolder(train_dir, transform=transform)
valid_data = ImageFolder(valid_dir, transform=transform)
test_data = ImageFolder(test_dir, transform=transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [5]:
#import torchinfo
#torchinfo.summary(model, input_size=(1, 3, 224, 224))

In [11]:
from model_architectures.v01 import HHP01
from model_architectures.v02 import HHP02
from model_architectures.AlexNet import AlexNet

# Initiate Model 
model = AlexNet(num_classes=3).to(device)
#model = HHP02(input_shape=3, hidden_units=10, num_classes=3).to(device)
#model = HHP01(input_shape=3, hidden_units=10, output_shape=3).to(device)

# Criterion and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.3)

# Training/Validation Loop
train_validate_model(num_epochs=100, train_loader=train_loader, valid_loader=valid_loader, model=model, criterion=criterion, optimizer=optimizer, device=device)

Epoch 1, Train Loss: 1.0982, Train Accuracy: 32.96%
Epoch 1, Validation Loss: 1.0988, Validation Accuracy: 30.56%
Epoch 2, Train Loss: 1.0975, Train Accuracy: 34.07%
Epoch 2, Validation Loss: 1.0988, Validation Accuracy: 30.56%
Epoch 3, Train Loss: 1.0986, Train Accuracy: 32.96%
Epoch 3, Validation Loss: 1.0989, Validation Accuracy: 30.56%
Epoch 4, Train Loss: 1.0993, Train Accuracy: 34.07%
Epoch 4, Validation Loss: 1.0988, Validation Accuracy: 30.56%
Epoch 5, Train Loss: 1.0983, Train Accuracy: 32.96%
Epoch 5, Validation Loss: 1.0989, Validation Accuracy: 30.56%
Epoch 6, Train Loss: 1.0987, Train Accuracy: 34.81%
Epoch 6, Validation Loss: 1.0989, Validation Accuracy: 30.56%
Epoch 7, Train Loss: 1.0989, Train Accuracy: 32.96%
Epoch 7, Validation Loss: 1.0988, Validation Accuracy: 30.56%
Epoch 8, Train Loss: 1.0980, Train Accuracy: 32.96%
Epoch 8, Validation Loss: 1.0989, Validation Accuracy: 30.56%
Epoch 9, Train Loss: 1.0992, Train Accuracy: 33.70%
Epoch 9, Validation Loss: 1.0989, Va

In [17]:
import torchvision.models as models
vgg11 = models.vgg11(weights='DEFAULT')
num_features = vgg11.classifier[6].in_features
vgg11.classifier[6] = nn.Linear(num_features, 3)

# Training/Validation Loop
train_validate_model(num_epochs=100, train_loader=train_loader, valid_loader=valid_loader, model=model, criterion=criterion, optimizer=optimizer, device=device)

Epoch 1, Train Loss: 1.0986, Train Accuracy: 33.33%
Epoch 1, Validation Loss: 1.0989, Validation Accuracy: 30.56%
Epoch 2, Train Loss: 1.0988, Train Accuracy: 33.70%
Epoch 2, Validation Loss: 1.0989, Validation Accuracy: 30.56%
Epoch 3, Train Loss: 1.0985, Train Accuracy: 34.07%
Epoch 3, Validation Loss: 1.0989, Validation Accuracy: 30.56%
Epoch 4, Train Loss: 1.0987, Train Accuracy: 33.70%
Epoch 4, Validation Loss: 1.0989, Validation Accuracy: 30.56%
Epoch 5, Train Loss: 1.0986, Train Accuracy: 33.70%
Epoch 5, Validation Loss: 1.0989, Validation Accuracy: 30.56%
Epoch 6, Train Loss: 1.0984, Train Accuracy: 33.33%
Epoch 6, Validation Loss: 1.0988, Validation Accuracy: 30.56%
Epoch 7, Train Loss: 1.0985, Train Accuracy: 33.70%
Epoch 7, Validation Loss: 1.0989, Validation Accuracy: 30.56%
Epoch 8, Train Loss: 1.0984, Train Accuracy: 33.70%
Epoch 8, Validation Loss: 1.0988, Validation Accuracy: 30.56%
Epoch 9, Train Loss: 1.0986, Train Accuracy: 33.70%
Epoch 9, Validation Loss: 1.0989, Va

In [ ]:
image_batch, label_batch = next(iter(train_loader))
image_batch.shape, label_batch.shape


In [18]:
from torchvision.models import resnet50, ResNet50_Weights
model = resnet50(weights=ResNet50_Weights.DEFAULT).to(device)
train_validate_model(num_epochs=100, train_loader=train_loader, valid_loader=valid_loader, model=model.softmax(3), criterion=criterion, optimizer=optimizer, device=device)
#num_features = resnet50.classifier[6].in_features
#vgg11.classifier[6] = nn.Linear(num_features, 3)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\MLDev/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:03<00:00, 31.6MB/s]


AttributeError: 'ResNet' object has no attribute 'softmax'